In [ ]:
# Imports here
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter 
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch import optim
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from PIL import Image


from PIL import Image
import json
from matplotlib.ticker import FormatStrFormatter

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
data_dir = './dataset'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])

trainset = torchvision.datasets.ImageFolder(root=train_dir, transform=train_transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=0)

testset = torchvision.datasets.ImageFolder(root=valid_dir, transform=test_transforms)
validloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=0)


In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 56 * 56, 512)  # Adjusted this line
        self.fc2 = nn.Linear(512, 102)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
class TripletDataset(Dataset):
    def __init__(self, data_dir, transform):
        self.data_dir = data_dir
        self.transform = transform
        self.labels = [item[1] for item in self.data_dir]

    def __len__(self):
        return len(self.data_dir)

    def __getitem__(self, idx):
        anchor, anchor_label = self.data_dir[idx]
        positive_idx = np.random.choice(len(self.data_dir))
        while self.labels[positive_idx] != anchor_label:
            positive_idx = np.random.choice(len(self.data_dir))
        positive, _ = self.data_dir[positive_idx]

        negative_idx = np.random.choice(len(self.data_dir))
        while self.labels[negative_idx] == anchor_label:
            negative_idx = np.random.choice(len(self.data_dir))
        negative, _ = self.data_dir[negative_idx]

        anchor = Image.fromarray(anchor.mul(255).byte().numpy().transpose((1, 2, 0)))
        positive = Image.fromarray(positive.mul(255).byte().numpy().transpose((1, 2, 0)))
        negative = Image.fromarray(negative.mul(255).byte().numpy().transpose((1, 2, 0)))

        anchor = self.transform(anchor)
        positive = self.transform(positive)
        negative = self.transform(negative)

        return anchor, positive, negative


In [ ]:
# Create triplet data loaders
triplet_trainset = TripletDataset(trainset, train_transforms)
triplet_trainloader = DataLoader(triplet_trainset, batch_size=32, shuffle=True, num_workers=0)
validloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=0)

criterion = nn.CrossEntropyLoss()


In [ ]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        distance_positive = (anchor - positive).pow(2).sum(1)
        distance_negative = (anchor - negative).pow(2).sum(1)
        loss = torch.clamp(distance_positive - distance_negative + self.margin, min=0.0).mean()
        return loss

# Create the CNN model
cnn_model = CNNModel()
cnn_model.to(device)

# Define the optimizer and loss function
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)
triplet_loss = TripletLoss()

In [ ]:
num_epochs = 100


# Lists to store training and validation losses and accuracies
train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []

model_checkpoint_path = 'triplet_loss_model.pth'
# ...

# Inside your training loop
for epoch in range(num_epochs):
    cnn_model.train()
    running_loss = 0.0

    # ...

    for i, data in enumerate(tqdm(triplet_trainloader), 0):
        anchor, positive, negative = data
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        optimizer.zero_grad()

        anchor_output = cnn_model(anchor)
        positive_output = cnn_model(positive)
        negative_output = cnn_model(negative)

        loss = triplet_loss(anchor_output, positive_output, negative_output)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_losses.append(running_loss / (i + 1))

    torch.save(cnn_model.state_dict(), model_checkpoint_path)

    # Calculate training accuracy based on the margin
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for i, data in enumerate(tqdm(triplet_trainloader), 0):
            anchor, positive, negative = data
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
            
            anchor_output = cnn_model(anchor)
            positive_output = cnn_model(positive)
            negative_output = cnn_model(negative)
            
            # Calculate the margin-based accuracy
            distance_positive = (anchor_output - positive_output).pow(2).sum(1)
            distance_negative = (anchor_output - negative_output).pow(2).sum(1)
            correct_train += torch.sum(distance_positive < distance_negative).item()
            total_train += anchor.size(0)

    train_accuracies.append(correct_train / total_train)
   
   #validation loop using cross enthropy 

    cnn_model.eval()
    running_loss = 0.0
    correct_valid = 0
    total_valid = 0

    with torch.no_grad():
        for i, data in enumerate(validloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = cnn_model(inputs)
            loss = criterion(outputs, labels)  # Use Cross-Entropy Loss

            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total_valid += labels.size(0)
            correct_valid += (predicted == labels).sum().item()

    valid_losses.append(running_loss / (i + 1))
    valid_accuracies.append(correct_valid / total_valid)

    print(f"Epoch {epoch + 1}, Loss: {train_losses[-1]}, Training Acc: {train_accuracies[-1]}, Validation Acc: {valid_accuracies[-1]}")

# ...

# Create a DataFrame
df = pd.DataFrame({
    'Epoch': range(1, num_epochs + 1),
    'Training Loss': train_losses,
    'Training Accuracy': train_accuracies,
    'Validation Loss': valid_losses,
    'Validation Accuracy': valid_accuracies
})

# Save the DataFrame to a CSV file
csv_file = 'triplet_loss_function_metrics.csv'
df.to_csv(csv_file, index=False)

In [ ]:
# After training, you can plot the training and validation losses and accuracies
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(valid_losses, label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(valid_accuracies, label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.show()

In [ ]:

# Create an instance of the CNNModel
model = CNNModel()

# Load the trained model's state dictionary
model.load_state_dict(torch.load('triplet_loss_model.pth'))
model.to(device)

# Switch the model to evaluation mode
model.eval()


In [ ]:
# Load the model
model = CNNModel()
model.load_state_dict(torch.load('triplet_loss_model.pth'))
model.to(device)

# Print the model architecture
print(model)


In [ ]:
# Load the model
model = CNNModel()
model.load_state_dict(torch.load('triplet_loss_model.pth'))
model.to(device)

# Access the number of output features in the final classification layer (fc2)
num_classes = model.fc2.out_features

# Print the number of classes
print("Number of classes:", num_classes)


In [ ]:
# Load the mapping from category names to labels
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Assuming you have already loaded the model and accessed the number of classes
model = CNNModel()
model.load_state_dict(torch.load('triplet_loss_model.pth'))
model.to(device)

num_classes = model.fc2.out_features

# Print out all the class names
for class_idx in range(num_classes):
    class_name = cat_to_name.get(str(class_idx), 'Unknown')
    print(f"Class {class_idx}: {class_name}")

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os 

image_path = "C:/Users/RZG_TESTER/Documents/GitHub/SC4001-Assignment-2/dataset/test/image_00005.jpg"

image = Image.open(image_path)

# Apply the same transformations used during training
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
image = transform(image).unsqueeze(0)  # Add a batch dimension

# Move the image to the same device as the model
image = image.to(device)

# Make a prediction using your model
model.eval()
with torch.no_grad():
    output = model(image)

# Get the predicted class
_, predicted_class = output.max(1)

# Map the predicted class index to its label
predicted_label = cat_to_name.get(str(predicted_class.item()), 'Unknown')

# Move the image tensor to the CPU for NumPy conversion
image = image.cpu()

# Display the image and its predicted class label
plt.imshow(np.array(image.squeeze().permute(1, 2, 0)))
plt.title(f'Predicted Class: {predicted_label}')
plt.axis('off')
plt.show()


In [ ]:
import os
import random

# Define the directory path for the test images
test_dir = 'C:/Users/RZG_TESTER/Documents/GitHub/SC4001-Assignment-2/dataset/test'

# List all the image files in the test directory
image_files = [os.path.join(test_dir, filename) for filename in os.listdir(test_dir) if filename.endswith(('.jpg', '.png', '.jpeg'))]

# Randomly select 5 images
random_images = random.sample(image_files, 5)

# Create a subplot with 1 row and 5 columns
fig, axes = plt.subplots(1, 5, figsize=(20, 5))

# Function to display an image with its predicted class
def display_image(image_path, ax):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Apply the same transformations
    image = image.to(device)

    model.eval()
    with torch.no_grad():
        output = model(image)

    _, predicted_class = output.max(1)
    predicted_label = cat_to_name.get(str(predicted_class.item()), 'Unknown')

    image = image.cpu()
    ax.imshow(np.array(image.squeeze().permute(1, 2, 0)))
    ax.set_title(f'Predicted Class: {predicted_label}')
    ax.axis('off')

# Display the randomly selected images side by side
for i, image_path in enumerate(random_images):
    display_image(image_path, axes[i])

plt.show()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def predict(image_path, model_path, topk=5):
    image = Image.open(image_path).convert("RGB")  # Remove '.jpg' and add .convert("RGB")

    # Define the transformation for the input image
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    image = preprocess(image).unsqueeze(0).to(device)  # Add .to(device)

    # Load the model and send it to the same device
    model = torch.load(model_path).to(device)

    # Ensure the model is in evaluation mode
    model.eval()

    # Forward pass to get probabilities
    with torch.no_grad():
        output = model(image)

    # Calculate probabilities and top class indices
    probs, indices = torch.topk(output, topk)

    # Convert indices to class labels
    classes = [str(idx.item()) for idx in indices[0]]

    return probs[0].cpu().tolist(), classes  # Add .cpu() to move results back to the CPU


In [ ]:
import os
import random

model_path = 'C:/Users/RZG_TESTER/Documents/GitHub/SC4001-Assignment-2/few_shot_learning_model.pth'
test_dir = data_dir + '/test'

# List all files in the test directory
test_files = []
for root, dirs, files in os.walk(test_dir):
    for file in files:
        if file.endswith(".jpg"):
            test_files.append(os.path.join(root, file).replace('\\', '/'))  # Use double backslashes

# Randomly select an image from the test directory
random_image_path = random.choice(test_files)

print(random_image_path)

# Getting prediction
probs, classes = predict(random_image_path, model_path, topk=5)

# Uncomment to check if the class of the flower (as seen from the file path) matches the top one predicted:
# print(classes[0])

# Converting classes to names
names = [cat_to_name[i] for i in classes]

# Creating PIL image
image = Image.open(random_image_path)

# Plotting the random test image and predicted probabilities
f, ax = plt.subplots(2, figsize=(6, 10))

ax[0].imshow(image)
ax[0].set_title(names[0])

y_names = np.arange(len(names))
ax[1].barh(y_names, probs, color='darkblue')
ax[1].set_yticks(y_names)
ax[1].set_yticklabels(names)
ax[1].invert_yaxis()

plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Get the model's parameters
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = [p.numel() for p in model_parameters]

# Create labels for each layer
layer_labels = [f'Layer {i}' for i in range(1, len(params) + 1)]

# Create a bar graph
plt.figure(figsize=(10, 5))
plt.bar(layer_labels, params)
plt.xlabel('Model Layers')
plt.ylabel('Number of Parameters')
plt.title('Number of Parameters in Each Layer')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()

# Show the graph
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Set the model to evaluation mode
model.eval()

# Initialize variables to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the validation dataset to collect labels
with torch.no_grad():
    for images, labels in validloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Convert the confusion matrix to a Pandas DataFrame for visualization
cm_df = pd.DataFrame(cm, index=cat_to_name.values(), columns=cat_to_name.values())

# Create a heatmap of the confusion matrix
plt.figure(figsize=(30, 8))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
